# Install Required Libraries

This cell ensures that the necessary Python libraries are installed.  
- `requests` is used for making HTTP requests to the ArcGIS REST API.
- `folium` is used for displaying geocoded points on an interactive map.

If you are running this notebook for the first time, this cell will install any missing packages.

In [ ]:
# Install the requests and folium libraries if you don't have them already
!pip install requests folium pyturf

# Import Libraries

This cell imports all the Python libraries needed for the workflow:
- `json` for working with JSON data.
- `requests` for making HTTP requests.
- `time` for measuring how long the geocoding takes.
- `folium` for mapping the geocoded results.

In [17]:
# Import required libraries
import folium    # For creating interactive maps
import json      # For handling JSON data
import requests  # For making HTTP requests
import time      # For measuring elapsed time
import turf    # pyturf is a Python port of turf.js for geospatial analysis

# Geocoding Addresses Using ArcGIS REST API (No Authentication Required)

This notebook demonstrates how to:
- Prepare a list of addresses for geocoding.
- Submit them to an ArcGIS REST geocoding service using Python.
- Save the results as a GeoJSON file.
- Display the results on an interactive map.

No authentication is required for the example service used here.

# Geocoding Addresses Using ArcGIS REST API (No Authentication Required)

This example shows how to submit up to 20 addresses to an ArcGIS REST geocoding service using Python's `requests` library. The code is fully commented for beginners.

# Submit Addresses for Geocoding

This cell prepares a list of addresses, formats them for the ArcGIS REST API, sends them to the geocoding service, and saves the results as a GeoJSON file in the `/data` directory.

**Key steps:**
- Set the geocoding service URL.
- Prepare up to 20 addresses in a list of dictionaries.
- Format the addresses as required by the API.
- Make a POST request to the service.
- Parse and print the results.
- Convert the results to GeoJSON and save to `/data/geocode_output.geojson`.

In [18]:
# 1. Set your ArcGIS geocoding service URL (no authentication needed)
geocode_service_url = "https://locator.stanford.edu/arcgis/rest/services/geocode/USA/GeocodeServer/geocodeAddresses"

# 2. Prepare a list of addresses (up to 20 for this example)
# Each address is a dictionary with at least an OBJECTID, Address, and City
addresses = [
    {"OBJECTID": 1, "Address": "REDWOOD AVE, MO / MELROSE ST, MO ( 42/B4 )", "City": "MODESTO"},
    {"OBJECTID": 2, "Address": "2504 DANA LN", "City": "MODESTO"},
    {"OBJECTID": 3, "Address": "1020 JAYHAWK WAY", "City": "MODESTO"}
    # Add more addresses here, up to 20 total
]

# 3. Format the addresses as required by the ArcGIS REST API
# The API expects a JSON object with a key "records", each containing an "attributes" dictionary
records = [{"attributes": addr} for addr in addresses]
addresses_json = {"records": records}

# 4. Set up the parameters for the POST request
params = {
    "addresses": json.dumps(addresses_json),  # Convert addresses to a JSON string
    "f": "json"  # Ask for the response in JSON format
}

# 5. Submit the request to the geocoding service
start_time = time.time()  # Start timing the request
response = requests.post(geocode_service_url, data=params)  # Send the POST request
elapsed = time.time() - start_time  # Calculate how long the request took

# 6. Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    geocode_results = response.json()
    # Print the geocoded results in a readable format
    print(json.dumps(geocode_results, indent=2))
    print(f"\nGeocoding completed in {elapsed:.2f} seconds.")

    # 7. Convert results to GeoJSON and save to file
    # Each location will become a GeoJSON Feature
    features = []
    for loc in geocode_results.get("locations", []):
        attrs = loc.get("attributes", {})  # Properties for the feature
        geometry = loc.get("location", {})  # Geometry (x=longitude, y=latitude)
        # GeoJSON expects coordinates as [longitude, latitude]
        if "x" in geometry and "y" in geometry:
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [geometry["x"], geometry["y"]]
                },
                "properties": attrs
            }
            features.append(feature)
    geojson = {
        "type": "FeatureCollection",
        "features": features
    }
    # Save to file in the /data directory
    output_path = "data/geocode_output.geojson"
    with open(output_path, "w") as f:
        json.dump(geojson, f, indent=2)
    print(f"GeoJSON output saved to {output_path}")
else:
    print(f"Error: Received status code {response.status_code}")
    # Print the error message from the server
    print(response.text)

{
  "spatialReference": {
    "wkid": 4326,
    "latestWkid": 4326
  },
  "locations": [
    {
      "address": "1020 Jayhawk Way, Modesto, California, 95358",
      "location": {
        "x": -121.033955729117,
        "y": 37.649057064042
      },
      "score": 100,
      "attributes": {
        "ResultID": 3,
        "Status": "M",
        "Score": 100,
        "Match_addr": "1020 Jayhawk Way, Modesto, California, 95358",
        "LongLabel": "1020 Jayhawk Way, Modesto, CA, 95358, USA",
        "ShortLabel": "1020 Jayhawk Way",
        "Addr_type": "PointAddress",
        "Type": "",
        "PlaceName": "",
        "Place_addr": "1020 Jayhawk Way, Modesto, California, 95358",
        "Phone": "",
        "URL": "",
        "Rank": 20,
        "AddBldg": "",
        "AddNum": "1020",
        "AddNumFrom": "",
        "AddNumTo": "",
        "AddRange": "",
        "Side": "",
        "StPreDir": "",
        "StPreType": "",
        "StName": "Jayhawk",
        "StType": "Way",
    

# Display Geocoded Results on a Map

This cell loads the GeoJSON file created in the previous step and displays the geocoded points on an interactive folium map.

**Key steps:**
- Load the GeoJSON file from `/data/geocode_output.geojson`.
- Calculate the extent (bounding box) of the points to center and zoom the map.
- Add the points as a GeoJSON layer to the map.
- Display the map in the notebook.

In [22]:
# Display the geocoded points on a folium map, using pyturf's centroid for map centering and bbox for zoom

# Load the GeoJSON file containing the geocoded results
geojson_path = "data/geocode_output.geojson"
with open(geojson_path, "r") as f:
    geojson_data = json.load(f)

# Use pyturf to calculate the bounding box (bbox) and centroid of all features
# bbox: [min_lon, min_lat, max_lon, max_lat]
# centroid: GeoJSON Point feature
if geojson_data["features"]:
    bbox = turf.bbox(geojson_data)
    centroid_feature = turf.centroid(geojson_data)
    # Extract centroid coordinates (longitude, latitude)
    center_lon, center_lat = centroid_feature["geometry"]["coordinates"]
    m = folium.Map(location=[center_lat, center_lon], zoom_start=13)
    # Fit the map to the bounding box
    m.fit_bounds([[bbox[1], bbox[0]], [bbox[3], bbox[2]]])
else:
    # If there are no points, show a default location
    m = folium.Map(location=[37.65, -120.99], zoom_start=12)

# Add the GeoJSON layer to the map so the points are visible
folium.GeoJson(geojson_data, name="Geocoded Points").add_to(m)

# Display the interactive map in the notebook
m